In [1]:
import pandas as pd

### Westernized samples from [curatedMetagenomicsData](http://bioconductor.org/packages/release/data/experiment/html/curatedMetagenomicData.html) package

In [2]:
west_count = pd.read_csv("curated_metagenomics/data/westernized_count.csv", index_col=0)

Choosing randomly 100 samples

In [3]:
west_count = west_count.sample(40, axis=1, random_state=42)

### Non-westernized samples from [curatedMetagenomicsData](http://bioconductor.org/packages/release/data/experiment/html/curatedMetagenomicData.html) package

In [4]:
non_west_count = pd.read_csv("curated_metagenomics/data/non_westernized_count.csv", index_col=0)

In [5]:
non_west_count = non_west_count.sample(40, axis=1, random_state=42)

In [6]:
non_west_count.shape

(2579, 40)

### Sediment samples from Archaelogical sediments

In [7]:
sediment_count = pd.read_csv("PRJEB18629/data/sediment_count.csv", index_col=1).drop("Unnamed: 0", 1).sample(40, axis=1, random_state=42)

### Marsha's samples

In [8]:
import os

In [9]:
def get_samp_basename(samp_name):
    return("_".join(samp_name.split("_")[0:2]))

In [10]:
def read_sample(samp_name):
    return(pd.read_csv("wibowo_mp/data/"+samp_name, 
                       sep="\t", index_col="#SampleID")
           .rename(columns={'Metaphlan2_Analysis':get_samp_basename(samp_name)}))

In [11]:
wib_metadata = pd.read_csv("wibowo_mp/samples_metadata.txt", sep="\t", index_col=0)

In [12]:
wib_metadata['total_reads'] = wib_metadata["Number of reads (pair 1)"] +  wib_metadata['Number of reads (pair 2)']

In [13]:
soil_samp = ['Lib4_10','Lib4_11','Lib4_12']

In [14]:
wib_samp = [i for i in os.listdir("wibowo_mp/data/")]

In [15]:
wib = read_sample(wib_samp[0])

In [16]:
for i in wib_samp[1:]:
    wib = wib.merge(read_sample(i), left_index=True, right_index=True, how='outer')

In [17]:
wib = wib.fillna(0)

In [18]:
for col in wib.columns:
    wib[col] = wib[col]/100*wib_metadata.loc[col,'total_reads']

### Merging all samples

In [19]:
all_samp = (west_count.merge(non_west_count, left_index=True, right_index=True, how='outer')
            .merge(sediment_count, left_index=True, right_index=True, how='outer')
            .merge(wib, left_index=True, right_index=True, how='outer').fillna(0))

In [20]:
all_samp.shape

(6558, 134)

Keeping only Taxons that occur in more than 0.1% of samples

In [21]:
all_samp = all_samp.loc[all_samp.astype(bool).sum(axis=1) >= int(all_samp.shape[1]/1000),:]

Keeping only Taxons that have a abundance greater than 0.1% in more than one sample

In [22]:
abundance_greater = all_samp/all_samp.sum(axis=0) > (0.1/100)

In [23]:
filtered_sample_abundance = abundance_greater.sum(axis=1) > 1

In [24]:
all_samp = all_samp.loc[filtered_sample_abundance,:]

Keeping only samples that have more 10000 mapping reads

In [25]:
all_samp = all_samp.loc[:,all_samp.sum(axis=0) >= 10000]

In [26]:
all_samp.shape

(450, 132)

### Preparing metadata 

In [27]:
meta_west = pd.Series(index=west_count.columns, data=['westernized_humans']*west_count.shape[1]).to_frame(name='Env')
meta_west['SourceSink'] = ["source"]*west_count.shape[1]

In [28]:
meta_non_west = pd.Series(index=non_west_count.columns, data=['non_westernized_humans']*non_west_count.shape[1]).to_frame(name='Env')
meta_non_west['SourceSink'] = ["source"]*non_west_count.shape[1]

In [29]:
meta_sediment = pd.Series(index=sediment_count.columns, data=['archeo_sediment']*sediment_count.shape[1]).to_frame(name='Env')
meta_sediment['SourceSink'] = ["source"]*sediment_count.shape[1]

In [30]:
meta_wib_sink = pd.Series(index=wib.columns, data=["-"]*wib.shape[1]).to_frame(name='Env')
meta_wib_sink['SourceSink'] = ['sink']*wib.shape[1]
meta_wib_sink.drop(soil_samp, axis=0, inplace=True)

In [31]:
meta_wib_source = pd.Series(index=soil_samp, data=['archeo_sediment']*len(soil_samp)).to_frame(name="Env")
meta_wib_source['SourceSink'] = ['source']*len(soil_samp)

In [32]:
meta = meta_west.append(meta_non_west).append(meta_wib_sink).append(meta_sediment).append(meta_wib_source)

In [33]:
meta = meta[['SourceSink','Env']]

Checking for concordance between metadata and data

In [34]:
all_samp=all_samp[meta.index.intersection(all_samp.T.index)]

In [35]:
all_samp = all_samp.astype(int)

In [36]:
meta = meta.loc[meta.index.intersection(all_samp.T.index),:]

In [37]:
meta.Env.value_counts()

archeo_sediment           41
westernized_humans        40
non_westernized_humans    40
-                         11
Name: Env, dtype: int64

### Exporting to csv

In [38]:
all_samp.to_csv("all_samp_st2.tsv", sep="\t", index_label='Taxon', )

In [39]:
meta.to_csv("labels_st2.tsv", sep="\t", index_label='#SampleID')

### Converting counts file to biom format for [SourceTracker2](https://github.com/biota/sourcetracker2)

In [40]:
!biom convert -i all_samp_st2.tsv -o all_samp_st2.biom --table-type="Taxon table" --to-hdf5

Minimum read count for SourceTracker rarefaction depth

In [41]:
all_samp.sum(axis=0).min()

11148

### Preparing data for [SourcePredict](https://github.com/maxibor/sourcepredict)

In [42]:
import re
import ete3

In [104]:
west_count = pd.read_csv("curated_metagenomics/data/westernized_count.csv", index_col=0).sample(100, axis=1, random_state=42)

In [105]:
non_west_count = pd.read_csv("curated_metagenomics/data/non_westernized_count.csv", index_col=0).sample(100, axis=1, random_state=42)

In [106]:
sediment_count = pd.read_csv("PRJEB18629/data/sediment_count.csv", index_col=1).drop("Unnamed: 0", 1)

In [139]:
taxon_counts = (west_count.merge(non_west_count, left_index=True, right_index=True, how='outer')
               .merge(sediment_count, left_index=True, right_index=True, how='outer')
               .merge(wib, left_index=True, right_index=True, how='outer').fillna(0))

In [109]:
regex = re.compile(".*s__(\w*)\.*")

In [110]:
for i in list(taxon_counts.index):
    gen = re.findall(regex, i)
    if len(gen) > 0:
        gen = gen[0]
    else:
        gen = ""
    taxon_counts.loc[i,'species'] = gen

In [111]:
taxon_counts['species'] = taxon_counts.species.str.replace("_", " ")

In [112]:
taxon_counts= taxon_counts.groupby("species").sum().drop("")

In [113]:
from ete3 import NCBITaxa
ncbi = NCBITaxa()

In [114]:
taxkey = pd.Series({k:v[0] for (k, v) in ncbi.get_name_translator(taxon_counts.index).items()}, name="TAXID")

In [115]:
def check_bacterial_species(taxid):
    rnk = list(ncbi.get_rank([taxid]).values())
    is_bacteria = ('Bacteria' in ncbi.get_taxid_translator(ncbi.get_lineage(taxid)).values())
    if len(rnk) == 1 and rnk[0] == 'species' and is_bacteria:
        return(True)
    else:
        return(False)

In [116]:
taxkey=taxkey.loc[taxkey.apply(check_bacterial_species)]

In [117]:
taxid_count = taxon_counts.merge(taxkey, left_index=True, right_index=True)

In [118]:
taxid_count = taxid_count.set_index("TAXID", drop=True)

In [119]:
taxid_count = taxid_count.loc[:,taxid_count.sum(axis=0) > 1]

In [120]:
#taxid_count = taxid_count.loc[taxid_count.astype(bool).sum(axis=1) > 2, :]

### Subsample normalization

In [121]:
taxid_count = ((taxid_count/taxid_count.sum(axis=0)*10000)).fillna(0).astype(int)

In [122]:
taxid_count = taxid_count.loc[taxid_count.sum(axis=1)!=0,:]

In [123]:
sink_count = taxid_count.loc[:,meta_wib_sink.index]

In [124]:
source_count = taxid_count.drop(meta_wib_sink.index, axis=1)

Labels

In [125]:
meta_west2 = pd.Series(index=west_count.columns, data=['westernized_humans']*west_count.shape[1]).to_frame(name='Env')
meta_west2['SourceSink'] = ["source"]*west_count.shape[1]
meta_non_west2 = pd.Series(index=non_west_count.columns, data=['non_westernized_humans']*non_west_count.shape[1]).to_frame(name='Env')
meta_non_west2['SourceSink'] = ["source"]*non_west_count.shape[1]
meta_sediment2 = pd.Series(index=sediment_count.columns, data=['archeo_sediment']*sediment_count.shape[1]).to_frame(name='Env')
meta_sediment2['SourceSink'] = ["source"]*sediment_count.shape[1]
meta = meta_west2.append(meta_non_west2).append(meta_wib_sink).append(meta_sediment2).append(meta_wib_source)

In [134]:
labels = meta.query("SourceSink != 'sink'").drop("SourceSink", axis=1).rename(columns={'Env':'labels'})

In [127]:
labels = labels.loc[labels.index.intersection(source_count.T.index),:]

In [128]:
labels.to_csv("sourcepredict/wib_labels.csv", index_label='SAMPLE')

In [129]:
source_count=source_count[labels.index.intersection(source_count.T.index)]

In [130]:
source_count.shape

(494, 208)

In [131]:
labels.shape

(208, 1)

In [132]:
labels['labels'].value_counts()

westernized_humans        100
non_westernized_humans    100
archeo_sediment             8
Name: labels, dtype: int64

In [133]:
sink_count.to_csv("sourcepredict/wib_sink.csv", index_label="TAXID")
source_count.to_csv("sourcepredict/wib_source.csv", index_label="TAXID")